In [7]:
import pandas as pd
import numpy as np

In [8]:
# Read the compressed CSV file
# file_path = r"C:\Users\JennyDuan\OneDrive - Burning Glass Institute\BGI Projects\Rural Data Paper\usa_00051.csv.gz"
file_path = "usa_00051.csv.gz"

# Read the data
df = pd.read_csv(file_path, compression='gzip')

print(f"Data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
print(df.head())

Data shape: (15912393, 22)
Columns: ['YEAR', 'MULTYEAR', 'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER', 'STATEFIP', 'COUNTYICP', 'COUNTYFIP', 'PUMA', 'MET2013', 'STRATA', 'GQ', 'PERNUM', 'PERWT', 'AGE', 'EMPSTAT', 'EMPSTATD', 'LABFORCE', 'IND', 'INDNAICS']

First few rows:
   YEAR  MULTYEAR  SAMPLE  SERIAL       CBSERIAL  HHWT        CLUSTER  \
0  2023      2019  202303       1  2019010000088   2.0  2023000000013   
1  2023      2019  202303       2  2019010000096  14.0  2023000000023   
2  2023      2019  202303       3  2019010000153   4.0  2023000000033   
3  2023      2019  202303       4  2019010000198  17.0  2023000000043   
4  2023      2019  202303       5  2019010000205  11.0  2023000000053   

   STATEFIP  COUNTYICP  COUNTYFIP  ...  STRATA  GQ  PERNUM  PERWT  AGE  \
0         1          0          0  ...  260001   4       1    2.0   39   
1         1          0          0  ...   70001   3       1   14.0   21   
2         1        150         15  ...   80001   4       1   

In [9]:
df

,YEAR,MULTYEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,COUNTYICP,COUNTYFIP,...,STRATA,GQ,PERNUM,PERWT,AGE,EMPSTAT,EMPSTATD,LABFORCE,IND,INDNAICS
0,2023,2019,202303,1,2019010000088,2.0,2023000000013,1,0,0,...,260001,4,1,2.0,39,3,30,1,0,0
1,2023,2019,202303,2,2019010000096,14.0,2023000000023,1,0,0,...,70001,3,1,14.0,21,3,30,1,0,0
2,2023,2019,202303,3,2019010000153,4.0,2023000000033,1,150,15,...,80001,4,1,4.0,19,1,10,2,9160,8131
3,2023,2019,202303,4,2019010000198,17.0,2023000000043,1,150,15,...,80001,3,1,17.0,77,3,30,1,0,0
4,2023,2019,202303,5,2019010000205,11.0,2023000000053,1,970,97,...,280301,3,1,11.0,41,3,30,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15912388,2023,2023,202303,7086486,2023001457972,15.0,2023070864863,56,210,21,...,30056,1,1,14.0,53,1,10,2,7860,6111
15912389,2023,2023,202303,7086486,2023001457972,15.0,2023070864863,56,210,21,...,30056,1,2,12.0,66,3,30,1,7860,6111
15912390,2023,2023,202303,7086487,2023001458196,15.0,2023070864873,56,0,0,...,20056,1,1,14.0,63,1,10,2,8191,622M
15912391,2023,2023,202303,7086488,2023001459187,7.0,2023070864883,56,0,0,...,10056,1,1,8.0,63,3,30,1,0,0


In [10]:
# Basic dataset overview
print(f"Total records in dataset: {len(df):,}")

# COUNTYICP analysis  
countyicp_identified = df[df['COUNTYICP'] != 0]
countyicp_missing = df[df['COUNTYICP'] == 0]
print("COUNTYICP (ICPSR codes) Coverage:")
print(f"  Records with identifiable counties: {len(countyicp_identified):,} ({len(countyicp_identified)/len(df)*100:.1f}%)")
print(f"  Records with unidentifiable counties: {len(countyicp_missing):,} ({len(countyicp_missing)/len(df)*100:.1f}%)")
print(f"  Unique counties identified: {df['COUNTYICP'].nunique()-1}")  # -1 to exclude 0
print()

# Population coverage
total_pop = df['PERWT'].sum()
countyicp_pop = countyicp_identified['PERWT'].sum()
print(f"Population coverage (using PERWT):")
print(f"  Total population: {total_pop:,.0f}")
print(f"  COUNTYICP identified population: {countyicp_pop:,.0f} ({countyicp_pop/total_pop*100:.1f}%)")
print()



Total records in dataset: 15,912,393
COUNTYICP (ICPSR codes) Coverage:
  Records with identifiable counties: 9,961,492 (62.6%)
  Records with unidentifiable counties: 5,950,901 (37.4%)
  Unique counties identified: 129

Population coverage (using PERWT):
  Total population: 332,387,543
  COUNTYICP identified population: 225,979,438 (68.0%)



# Rural Coverage

In [28]:
# Create rural dataframe
rural_df = df[df['MET2013'] == 0].copy()
rural_df = rural_df[rural_df['PUMA'] != 0]  # Ensure PUMA is identified
rural_df['metro_status'] = 'Non-metro/Rural'

In [29]:
# Calculate PUMA populations in rural areas
print("Calculating rural PUMA populations...")
# Create unique PUMA identifier using STATEFIP + PUMA
rural_df['unique_puma'] = rural_df['STATEFIP'].astype(str) + '_' + rural_df['PUMA'].astype(str)

# Add metro status classification
rural_df['metro_status'] = 'Non-metro/Rural'

print(f"   Rural DataFrame: {len(rural_df):,} records")
print(f"   Rural population: {rural_df['PERWT'].sum():,.0f}")
print(f"   Rural PUMAs: {rural_df['PUMA'].nunique()}")

Calculating rural PUMA populations...
   Rural DataFrame: 4,142,396 records
   Rural population: 69,123,668
   Rural PUMAs: 164


In [30]:
rural_df

,YEAR,MULTYEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,COUNTYICP,COUNTYFIP,...,PERNUM,PERWT,AGE,EMPSTAT,EMPSTATD,LABFORCE,IND,INDNAICS,metro_status,unique_puma
0,2023,2019,202303,1,2019010000088,2.0,2023000000013,1,0,0,...,1,2.0,39,3,30,1,0,0,Non-metro/Rural,1_2600
1,2023,2019,202303,2,2019010000096,14.0,2023000000023,1,0,0,...,1,14.0,21,3,30,1,0,0,Non-metro/Rural,1_700
7,2023,2019,202303,8,2019010000284,3.0,2023000000083,1,0,0,...,1,3.0,35,3,30,1,0,0,Non-metro/Rural,1_1100
12,2023,2019,202303,13,2019010000617,14.0,2023000000133,1,0,0,...,1,14.0,19,3,30,1,0,0,Non-metro/Rural,1_2000
16,2023,2019,202303,17,2019010000742,16.0,2023000000173,1,0,0,...,1,16.0,25,3,30,1,0,0,Non-metro/Rural,1_1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15912386,2023,2023,202303,7086485,2023001456541,25.0,2023070864853,56,0,0,...,3,15.0,42,1,10,2,8680,722Z,Non-metro/Rural,56_500
15912387,2023,2023,202303,7086485,2023001456541,25.0,2023070864853,56,0,0,...,4,15.0,38,1,10,2,9480,923,Non-metro/Rural,56_500
15912390,2023,2023,202303,7086487,2023001458196,15.0,2023070864873,56,0,0,...,1,14.0,63,1,10,2,8191,622M,Non-metro/Rural,56_200
15912391,2023,2023,202303,7086488,2023001459187,7.0,2023070864883,56,0,0,...,1,8.0,63,3,30,1,0,0,Non-metro/Rural,56_100


In [31]:
# Rural pumas population counts
rural_puma_pop = rural_df.groupby('unique_puma').agg({
    'PERWT': 'sum',
    'STATEFIP': 'first', 
    'PUMA': 'first'
}).reset_index()
rural_puma_pop.columns = ['unique_puma', 'population', 'STATEFIP', 'PUMA']
print(f"Rural PUMAs identified: {len(rural_puma_pop)}")

Rural PUMAs identified: 509


In [32]:
# Create population bins
pop_bins = [0, 75000, 100000, 125000, 150000, 175000, 200000, 250000, np.inf]
pop_labels = ['0-75K', '75-100K', '100-125K', '125-150K', '150-175K', '175-200K', '200-250K', '250K+']
rural_puma_pop['pop_label'] = pd.cut(rural_puma_pop['population'], bins=pop_bins, labels=pop_labels, right=False)
puma_counts = rural_puma_pop['pop_label'].value_counts().sort_index()

In [33]:
# Get employed persons
rural_employed = rural_df[rural_df['EMPSTAT'] == 1].copy()
print(f"Total rural employed persons: {rural_employed['PERWT'].sum():,.0f}")

# Map to NAICS 2-digit
def map_to_naics_2digit(ind_code):
    """Map IND codes to 2-digit NAICS sectors"""
    if pd.isna(ind_code) or ind_code == 0:
        return 'Other'
    
    ind_str = str(int(ind_code)).zfill(4)
    
    if ind_str.startswith(('017', '018', '019', '027', '028', '029')):
        return 'NAICS_11'  # Agriculture, Forestry, Fishing and Hunting
    elif ind_str.startswith(('037', '038', '039', '047', '048', '049')):
        return 'NAICS_21'  # Mining, Quarrying, and Oil and Gas Extraction
    elif ind_str.startswith(('057', '058', '059')):
        return 'NAICS_22'  # Utilities
    elif ind_str.startswith(('067', '068', '069', '077', '078', '079')):
        return 'NAICS_23'  # Construction
    elif ind_str.startswith(('107', '108', '109', '117', '118', '119', '127', '128', '129',
                             '137', '138', '139', '147', '148', '149', '157', '158', '159',
                             '167', '168', '169', '177', '178', '179', '187', '188', '189',
                             '197', '198', '199', '207', '208', '209', '217', '218', '219',
                             '227', '228', '229', '237', '238', '239', '247', '248', '249',
                             '257', '258', '259', '267', '268', '269', '277', '278', '279',
                             '287', '288', '289', '297', '298', '299', '307', '308', '309',
                             '317', '318', '319', '327', '328', '329', '337', '338', '339',
                             '347', '348', '349', '357', '358', '359', '367', '368', '369',
                             '377', '378', '379', '387', '388', '389', '397', '398', '399')):
        return 'NAICS_31-33'  # Manufacturing
    elif ind_str.startswith(('407', '408', '409', '417', '418', '419', '427', '428', '429')):
        return 'NAICS_42'  # Wholesale Trade
    elif ind_str.startswith(('447', '448', '449', '457', '458', '459', '467', '468', '469',
                             '477', '478', '479', '487', '488', '489', '497', '498', '499',
                             '507', '508', '509', '517', '518', '519', '527', '528', '529',
                             '537', '538', '539', '547', '548', '549', '557', '558', '559',
                             '567', '568', '569', '577', '578', '579', '587', '588', '589',
                             '597', '598', '599')):
        return 'NAICS_44-45'  # Retail Trade
    elif ind_str.startswith(('607', '608', '609', '617', '618', '619', '627', '628', '629',
                             '637', '638', '639', '647', '648', '649', '657', '658', '659')):
        return 'NAICS_48-49'  # Transportation and Warehousing
    elif ind_str.startswith(('667', '668', '669', '677', '678', '679', '687', '688', '689')):
        return 'NAICS_51'  # Information
    elif ind_str.startswith(('697', '698', '699', '707', '708', '709')):
        return 'NAICS_52'  # Finance and Insurance
    elif ind_str.startswith(('717', '718', '719', '727', '728', '729')):
        return 'NAICS_53'  # Real Estate and Rental and Leasing
    elif ind_str.startswith(('737', '738', '739', '747', '748', '749', '757', '758', '759')):
        return 'NAICS_54'  # Professional, Scientific, and Technical Services
    elif ind_str.startswith(('767', '768', '769')):
        return 'NAICS_55'  # Management of Companies and Enterprises
    elif ind_str.startswith(('777', '778', '779', '787', '788', '789')):
        return 'NAICS_56'  # Administrative and Support and Waste Management
    elif ind_str.startswith(('807', '808', '809', '817', '818', '819')):
        return 'NAICS_61'  # Educational Services
    elif ind_str.startswith(('827', '828', '829', '837', '838', '839', '847', '848', '849',
                             '857', '858', '859', '867', '868', '869')):
        return 'NAICS_62'  # Health Care and Social Assistance
    elif ind_str.startswith(('877', '878', '879', '887', '888', '889')):
        return 'NAICS_71'  # Arts, Entertainment, and Recreation
    elif ind_str.startswith(('897', '898', '899')):
        return 'NAICS_72'  # Accommodation and Food Services
    elif ind_str.startswith(('907', '908', '909', '917', '918', '919', '927', '928', '929')):
        return 'NAICS_81'  # Other Services (except Public Administration)
    elif ind_str.startswith(('937', '938', '939', '947', '948', '949', '957', '958', '959')):
        return 'NAICS_92'  # Public Administration
    else:
        return 'Other'

# Apply NAICS mapping
rural_employed['naics_2digit'] = rural_employed['IND'].apply(map_to_naics_2digit)

# Add unique PUMA identifier to employed data
rural_employed['unique_puma'] = rural_employed['STATEFIP'].astype(str) + '_' + rural_employed['PUMA'].astype(str)

# Add population label to employed data
rural_employed_with_pop = rural_employed.merge(rural_puma_pop[['unique_puma', 'pop_label']], on='unique_puma', how='left')

Total rural employed persons: 31,199,345


In [34]:
# Calculate employment by population bucket and NAICS sector
employment_by_pop_naics = rural_employed_with_pop.groupby(['pop_label', 'naics_2digit'], observed=True)['PERWT'].sum().unstack(fill_value=0)

# Calculate total employment by population bucket
total_employment_by_pop = employment_by_pop_naics.sum(axis=1)

# Calculate employment percentages
employment_percentages = employment_by_pop_naics.div(total_employment_by_pop, axis=0) * 100

print("Employment percentages calculated")

# Calculate total population and total employed by population bucket
population_summary = rural_puma_pop.groupby('pop_label', observed=True).agg({
    'population': 'sum',
    'unique_puma': 'count'
}).rename(columns={'unique_puma': 'n_pumas'}).reset_index()

employed_summary = rural_employed_with_pop.groupby('pop_label', observed=True)['PERWT'].sum().reset_index()
employed_summary.columns = ['pop_label', 'total_employed']

# Merge population and employment data
pop_emp_summary = population_summary.merge(employed_summary, on='pop_label', how='left')
pop_emp_summary['total_employed'] = pop_emp_summary['total_employed'].fillna(0)

# Get ALL NAICS sectors (including Other)
all_sectors = [col for col in employment_percentages.columns if col.startswith('NAICS') or col == 'Other']

# Create complete summary with ALL sectors
complete_summary = employment_percentages[all_sectors].fillna(0).round(0).astype(int)
complete_summary = complete_summary.reset_index()

# Merge with population and employment totals  
complete_summary = complete_summary.merge(pop_emp_summary, on='pop_label', how='left')

# Move key columns to front
key_cols = ['pop_label', 'n_pumas', 'population', 'total_employed']
sector_cols = [col for col in complete_summary.columns if col.startswith('NAICS') or col == 'Other']
final_complete_cols = key_cols + sector_cols
complete_summary = complete_summary[final_complete_cols]

print("Complete employment summary created")


Employment percentages calculated
Complete employment summary created


In [35]:
# Calculate total employment by population bucket
total_employment = rural_employed_with_pop.groupby('pop_label', observed=True)['PERWT'].sum().reset_index()
total_employment.columns = ['pop_label', 'total_employment']

# Check NAICS 2-digit conservation
naics2_employment = rural_employed_with_pop.groupby(['pop_label', 'naics_2digit'], observed=True)['PERWT'].sum().reset_index()
naics2_totals = naics2_employment.groupby('pop_label', observed=True)['PERWT'].sum().reset_index()
naics2_totals.columns = ['pop_label', 'naics2_total']

# Create conservation check
conservation_check = total_employment.merge(naics2_totals, on='pop_label')
conservation_check['naics2_coverage_pct'] = (conservation_check['naics2_total'] / conservation_check['total_employment'] * 100).round(1)

In [36]:
# Create detailed NAICS mappings for other levels
def get_naics_levels(ind_code):
    if pd.isna(ind_code) or ind_code == 0:
        return {
            'naics_2digit_detail': 'Missing',
            'naics_3digit_detail': 'Missing', 
            'naics_4digit_detail': 'Missing',
            'naics_5digit_detail': 'Missing',
            'naics_6digit_detail': 'Missing'
        }
    
    ind_str = str(int(ind_code)).zfill(4)
    
    return {
        'naics_2digit_detail': ind_str[:1] + 'X',
        'naics_3digit_detail': ind_str[:2] + 'X',
        'naics_4digit_detail': ind_str[:3] + 'X',
        'naics_5digit_detail': ind_str[:4] + 'X',
        'naics_6digit_detail': ind_str[:4] + 'XX'
    }

# Apply to data
naics_levels = rural_employed_with_pop['IND'].apply(get_naics_levels)
naics_levels_df = pd.DataFrame(naics_levels.tolist())

for col in naics_levels_df.columns:
    rural_employed_with_pop[col] = naics_levels_df[col]

In [37]:
# Check conservation at all detail levels
naics_levels_to_check = ['naics_2digit_detail', 'naics_3digit_detail', 'naics_4digit_detail', 'naics_5digit_detail', 'naics_6digit_detail']
conservation_summary = total_employment.copy()

for level in naics_levels_to_check:
    level_employment = rural_employed_with_pop.groupby(['pop_label', level], observed=True)['PERWT'].sum().reset_index()
    level_totals = level_employment.groupby('pop_label', observed=True)['PERWT'].sum().reset_index()
    level_totals.columns = ['pop_label', f'{level}_total']
    
    conservation_summary = conservation_summary.merge(level_totals, on='pop_label', how='left')
    conservation_summary[f'{level}_coverage_pct'] = (
        conservation_summary[f'{level}_total'] / conservation_summary['total_employment'] * 100
    ).round(1)


In [38]:
# Add PUMA counts
puma_counts_df = puma_counts.reset_index()
puma_counts_df.columns = ['pop_label', 'n_pumas']
conservation_summary = conservation_summary.merge(puma_counts_df, on='pop_label', how='left')

In [39]:
# Create final conservation table
display_cols = ['pop_label', 'n_pumas', 'total_employment']
coverage_cols = [col for col in conservation_summary.columns if col.endswith('_coverage_pct')]
display_cols.extend(coverage_cols)

final_conservation_table = conservation_summary[display_cols].copy()
final_conservation_table['total_employment'] = final_conservation_table['total_employment'].round(0).astype(int)


In [40]:
complete_summary

,pop_label,n_pumas,population,total_employed,NAICS_11,NAICS_21,NAICS_22,NAICS_23,NAICS_31-33,NAICS_42,...,NAICS_54,NAICS_55,NAICS_56,NAICS_61,NAICS_62,NAICS_71,NAICS_72,NAICS_81,NAICS_92,Other
0,75-100K,3,299001.0,117644.0,8,2,1,9,5,1,...,3,1,3,10,12,2,1,1,8,12
1,100-125K,212,23947937.0,10832676.0,3,1,1,8,13,1,...,3,1,4,8,12,2,1,1,5,12
2,125-150K,163,22141412.0,10003378.0,3,1,1,8,13,1,...,3,1,5,8,12,2,1,1,5,12
3,150-175K,80,13052213.0,5901389.0,3,1,1,8,12,1,...,3,1,5,8,12,2,1,1,5,13
4,175-200K,45,8399756.0,3752598.0,2,1,1,8,12,1,...,3,1,5,9,12,2,1,1,5,12
5,200-250K,6,1283349.0,591660.0,2,0,1,8,8,1,...,4,2,5,7,12,1,1,1,5,16


In [24]:
final_conservation_table

,pop_label,n_pumas,total_employment,naics_2digit_detail_coverage_pct,naics_3digit_detail_coverage_pct,naics_4digit_detail_coverage_pct,naics_5digit_detail_coverage_pct,naics_6digit_detail_coverage_pct
0,75-100K,3,117644,100.0,100.0,100.0,100.0,100.0
1,100-125K,212,10832676,100.0,100.0,100.0,100.0,100.0
2,125-150K,163,10003378,100.0,100.0,100.0,100.0,100.0
3,150-175K,80,5901389,100.0,100.0,100.0,100.0,100.0
4,175-200K,45,3752598,100.0,100.0,100.0,100.0,100.0
5,200-250K,6,591660,100.0,100.0,100.0,100.0,100.0


In [41]:
# Calculate total employment by population bucket
print("Total employment by population bucket...")
total_employment = rural_employed_with_pop.groupby('pop_label')['PERWT'].sum().reset_index()
total_employment.columns = ['pop_label', 'total_employment']
print(total_employment)
print()

Total employment by population bucket...
  pop_label  total_employment
0     0-75K               0.0
1   75-100K          117644.0
2  100-125K        10832676.0
3  125-150K        10003378.0
4  150-175K         5901389.0
5  175-200K         3752598.0
6  200-250K          591660.0
7     250K+               0.0



/tmp/ipykernel_10619/63071298.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_employment = rural_employed_with_pop.groupby('pop_label')['PERWT'].sum().reset_index()


### Version 2 PUMAs rural and urban

In [46]:
# Create All PUMAs (Rural + Non-Rural)
print("Creating ALL PUMAs analysis (rural + non-rural)...")

# All PUMAs with PUMA identification
all_puma_df = df[df['PUMA'] != 0].copy()

# Add metro status classification
all_puma_df['metro_status'] = all_puma_df['MET2013'].apply(
    lambda x: 'Non-metro/Rural' if x == 0 else 'Metropolitan'
)

print(f"Total PUMAs data: {len(all_puma_df):,} records")
print(f"Total population: {all_puma_df['PERWT'].sum():,.0f}")

# Check metro status distribution
metro_distribution = all_puma_df.groupby('metro_status')['PERWT'].sum()
for status, pop in metro_distribution.items():
    pct = pop / metro_distribution.sum() * 100
    print(f"  {status}: {pop:,.0f} ({pct:.1f}%)")
print()


Creating ALL PUMAs analysis (rural + non-rural)...
Total PUMAs data: 15,912,393 records
Total population: 332,387,543
  Metropolitan: 263,263,875 (79.2%)
  Non-metro/Rural: 69,123,668 (20.8%)



In [47]:
# PUMA Population Analysis for ALL PUMAs
print("PUMA population analysis for ALL areas...")

# Create unique PUMA identifier
all_puma_df['unique_puma'] = all_puma_df['STATEFIP'].astype(str) + '_' + all_puma_df['PUMA'].astype(str)

# Calculate PUMA populations by metro status
all_puma_pop = all_puma_df.groupby('unique_puma').agg({
    'PERWT': 'sum',
    'STATEFIP': 'first',
    'PUMA': 'first',
    'MET2013': 'first'
}).reset_index()
all_puma_pop.columns = ['unique_puma', 'population', 'STATEFIP', 'PUMA', 'MET2013']

# Add metro status
all_puma_pop['metro_status'] = all_puma_pop['MET2013'].apply(
    lambda x: 'Non-metro/Rural' if x == 0 else 'Metropolitan'
)

# Create population bins
pop_bins = [0, 75000, 100000, 125000, 150000, 175000, 200000, 250000, 300000, 400000, 500000, np.inf]
pop_labels = ['0-75K', '75-100K', '100-125K', '125-150K', '150-175K', '175-200K', 
              '200-250K', '250-300K', '300-400K', '400-500K', '500K+']

all_puma_pop['pop_label'] = pd.cut(all_puma_pop['population'], bins=pop_bins, labels=pop_labels, right=False)

print(f"Total PUMAs: {len(all_puma_pop)}")
print(f"Rural PUMAs: {(all_puma_pop['metro_status'] == 'Non-metro/Rural').sum()}")
print(f"Metro PUMAs: {(all_puma_pop['metro_status'] == 'Metropolitan').sum()}")
print()

# PUMAs by population size and metro status
puma_counts_by_metro = all_puma_pop.groupby(['pop_label', 'metro_status']).size().unstack(fill_value=0)
print("PUMAs by population size and metro status:")
print(puma_counts_by_metro)
print()

PUMA population analysis for ALL areas...
Total PUMAs: 2462
Rural PUMAs: 509
Metro PUMAs: 1953

PUMAs by population size and metro status:
metro_status  Metropolitan  Non-metro/Rural
pop_label                                  
0-75K                    0                0
75-100K                 31                3
100-125K               846              212
125-150K               565              163
150-175K               328               80
175-200K               154               45
200-250K                28                6
250-300K                 1                0
300-400K                 0                0
400-500K                 0                0
500K+                    0                0



/tmp/ipykernel_10619/1995347115.py:34: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  puma_counts_by_metro = all_puma_pop.groupby(['pop_label', 'metro_status']).size().unstack(fill_value=0)


In [50]:
# Employment Analysis for ALL PUMAs
print("Employment analysis for ALL PUMAs...")

# Get employed persons from all PUMAs
all_employed = all_puma_df[all_puma_df['EMPSTAT'] == 1].copy()
print(f"Total employed persons: {all_employed['PERWT'].sum():,.0f}")

# Apply NAICS mapping (same function as before)
def map_to_naics_2digit(ind_code):
    if pd.isna(ind_code) or ind_code == 0:
        return 'Other'
    
    ind_str = str(int(ind_code)).zfill(4)
    
    if ind_str.startswith(('017', '018', '019', '027', '028', '029')):
        return 'NAICS_11'
    elif ind_str.startswith(('067', '068', '069', '077', '078', '079')):
        return 'NAICS_23'
    elif ind_str.startswith(('107', '108', '109', '117', '118', '119', '127', '128', '129',
                             '137', '138', '139', '147', '148', '149', '157', '158', '159',
                             '167', '168', '169', '177', '178', '179', '187', '188', '189',
                             '197', '198', '199', '207', '208', '209', '217', '218', '219',
                             '227', '228', '229', '237', '238', '239', '247', '248', '249',
                             '257', '258', '259', '267', '268', '269', '277', '278', '279',
                             '287', '288', '289', '297', '298', '299', '307', '308', '309',
                             '317', '318', '319', '327', '328', '329', '337', '338', '339',
                             '347', '348', '349', '357', '358', '359', '367', '368', '369',
                             '377', '378', '379', '387', '388', '389', '397', '398', '399')):
        return 'NAICS_31-33'
    elif ind_str.startswith(('447', '448', '449', '457', '458', '459', '467', '468', '469',
                             '477', '478', '479', '487', '488', '489', '497', '498', '499',
                             '507', '508', '509', '517', '518', '519', '527', '528', '529',
                             '537', '538', '539', '547', '548', '549', '557', '558', '559',
                             '567', '568', '569', '577', '578', '579', '587', '588', '589',
                             '597', '598', '599')):
        return 'NAICS_44-45'
    elif ind_str.startswith(('827', '828', '829', '837', '838', '839', '847', '848', '849',
                             '857', '858', '859', '867', '868', '869')):

        return 'NAICS_62'
    else:
        return 'Other'

all_employed['naics_2digit'] = all_employed['IND'].apply(map_to_naics_2digit)
all_employed['unique_puma'] = all_employed['STATEFIP'].astype(str) + '_' + all_employed['PUMA'].astype(str)

# Merge with PUMA population data
all_employed_clean = all_employed.drop('metro_status', axis=1)  # Remove from left dataframe
all_employed_with_pop = all_employed_clean.merge(
    all_puma_pop[['unique_puma', 'pop_label', 'metro_status']], 
    on='unique_puma', how='left'
)

print("Employment by metro status:")
emp_by_metro = all_employed_with_pop.groupby('metro_status')['PERWT'].sum()
for status, emp in emp_by_metro.items():
    pct = emp / emp_by_metro.sum() * 100
    print(f"  {status}: {emp:,.0f} ({pct:.1f}%)")
print()


Employment analysis for ALL PUMAs...
Total employed persons: 160,913,396
Employment by metro status:
  Metropolitan: 129,714,051 (80.6%)
  Non-metro/Rural: 31,199,345 (19.4%)



In [53]:
# Create separate coverage analysis by metro status
def create_coverage_by_metro_status(employed_data, puma_data, metro_type):
    """Create coverage analysis for specific metro status"""
    
    # Filter data
    metro_employed = employed_data[employed_data['metro_status'] == metro_type]
    metro_pumas = puma_data[puma_data['metro_status'] == metro_type]
    
    if len(metro_employed) == 0:
        print(f"No data for {metro_type}")
        return pd.DataFrame(), pd.DataFrame()
    
    print(f"Analyzing {metro_type}: {len(metro_employed):,} employment records")
    
    # Apply detailed NAICS mapping if not already done
    if 'naics_2digit' not in metro_employed.columns:
        def create_naics_detail_levels(ind_code):
            if pd.isna(ind_code) or ind_code == 0:
                return {
                    'naics_2digit': 'Unclassified',
                    'naics_3digit': 'Unclassified',
                    'naics_4digit': 'Unclassified', 
                    'naics_5digit': 'Unclassified',
                    'naics_6digit': 'Unclassified'
                }
            
            ind_str = str(int(ind_code)).zfill(4)
            return {
                'naics_2digit': ind_str[0] + '0',
                'naics_3digit': ind_str[:2] + '0',
                'naics_4digit': ind_str[:3] + '0',
                'naics_5digit': ind_str,
                'naics_6digit': ind_str + str(ind_code % 10)
            }
        
        naics_detail_mapping = metro_employed['IND'].apply(create_naics_detail_levels)
        naics_detail_df = pd.DataFrame(naics_detail_mapping.tolist())
        
        for col in naics_detail_df.columns:
            metro_employed[col] = naics_detail_df[col]
    
    # Calculate coverage for each NAICS level
    naics_levels = ['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']
    coverage_results = {}
    
    for level in naics_levels:
        coverage_by_pop = []
        
        # Only analyze population buckets that actually exist
        existing_pop_buckets = metro_employed['pop_label'].dropna().unique()
        
        for pop_bucket in existing_pop_buckets:
            bucket_data = metro_employed[metro_employed['pop_label'] == pop_bucket]
            
            total_employment = bucket_data['PERWT'].sum()
            classifiable_employment = bucket_data[bucket_data[level] != 'Unclassified']['PERWT'].sum()
            coverage_pct = (classifiable_employment / total_employment * 100) if total_employment > 0 else 0
            
            coverage_by_pop.append({
                'pop_label': pop_bucket,
                'level': level,
                'coverage_percent': coverage_pct,
                'metro_status': metro_type
            })
        
        coverage_results[level] = pd.DataFrame(coverage_by_pop)
    
    # Combine coverage results
    all_coverage = pd.concat(coverage_results.values(), ignore_index=True)
    
    # Pivot to get coverage by level and population bucket
    coverage_pivot = all_coverage.pivot(index='pop_label', columns='level', values='coverage_percent').round(1)
    
    # Add PUMA counts (keep all population buckets, show 0 for empty ones)
    puma_counts = metro_pumas['pop_label'].value_counts().sort_index()
    
    # Create complete population bucket list (including empty ones)
    all_possible_buckets = pd.Index(['0-75K', '75-100K', '100-125K', '125-150K', '150-175K', 
                                   '175-200K', '200-250K', '250-300K', '300-400K', '400-500K', '500K+'])
    puma_counts = puma_counts.reindex(all_possible_buckets, fill_value=0)
    
    puma_counts_df = puma_counts.reset_index()
    puma_counts_df.columns = ['pop_label', 'n_pumas']
    
    # Merge and fill coverage values
    coverage_pivot = coverage_pivot.reset_index().merge(puma_counts_df, on='pop_label', how='right')
    
    # Fill NaN coverage values with 0 for population buckets with no PUMAs
    naics_levels = ['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']
    for level in naics_levels:
        coverage_pivot[level] = coverage_pivot[level].fillna(0)
    
    # Reorder columns and sort by population bucket
    cols = ['pop_label', 'n_pumas'] + naics_levels
    coverage_pivot = coverage_pivot[cols]
    
    # Sort by population bucket order
    bucket_order = ['0-75K', '75-100K', '100-125K', '125-150K', '150-175K', 
                   '175-200K', '200-250K', '250-300K', '300-400K', '400-500K', '500K+']
    coverage_pivot['pop_label'] = pd.Categorical(coverage_pivot['pop_label'], categories=bucket_order, ordered=True)
    coverage_pivot = coverage_pivot.sort_values('pop_label').reset_index(drop=True)
    
    return coverage_pivot, puma_counts


In [54]:
# Create coverage for Rural, Urban, and Combined
print("Creating coverage analysis for rural areas...")
rural_coverage, rural_puma_counts = create_coverage_by_metro_status(
    all_employed_with_pop, all_puma_pop, 'Non-metro/Rural'
)

print("Creating coverage analysis for urban areas...")
urban_coverage, urban_puma_counts = create_coverage_by_metro_status(
    all_employed_with_pop, all_puma_pop, 'Metropolitan'
)

Creating coverage analysis for rural areas...
Analyzing Non-metro/Rural: 1,766,302 employment records


KeyError: 'naics_3digit'

In [ ]:


# STEP 2: Create coverage for Rural, Urban, and Combined
print("Creating coverage analysis for rural areas...")
rural_coverage, rural_puma_counts = create_coverage_by_metro_status(
    all_employed_with_pop, all_puma_pop, 'Non-metro/Rural'
)

print("Creating coverage analysis for urban areas...")
urban_coverage, urban_puma_counts = create_coverage_by_metro_status(
    all_employed_with_pop, all_puma_pop, 'Metropolitan'
)

# STEP 3: Create combined total with rural/urban breakdown
print("Creating combined totals with rural/urban breakdown...")

def create_combined_coverage_with_split():
    """Create coverage analysis showing rural/urban split within each population bucket"""
    
    naics_levels = ['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']
    
    # Create comprehensive breakdown
    coverage_breakdown = []
    
    all_possible_buckets = ['0-75K', '75-100K', '100-125K', '125-150K', '150-175K', 
                           '175-200K', '200-250K', '250-300K', '300-400K', '400-500K', '500K+']
    
    for pop_bucket in all_possible_buckets:
        # Get data for this population bucket
        bucket_data = all_employed_with_pop[all_employed_with_pop['pop_label'] == pop_bucket]
        
        if len(bucket_data) == 0:
            # No data for this bucket - add zeros
            coverage_breakdown.append({
                'pop_label': pop_bucket,
                'total_pumas': 0,
                'rural_pumas': 0,
                'urban_pumas': 0,
                **{f'total_{level}': 0.0 for level in naics_levels},
                **{f'rural_{level}': 0.0 for level in naics_levels},
                **{f'urban_{level}': 0.0 for level in naics_levels}
            })
            continue
        
        # Count PUMAs by metro status
        puma_counts_by_metro = all_puma_pop[all_puma_pop['pop_label'] == pop_bucket]['metro_status'].value_counts()
        total_pumas = puma_counts_by_metro.sum()
        rural_pumas = puma_counts_by_metro.get('Non-metro/Rural', 0)
        urban_pumas = puma_counts_by_metro.get('Metropolitan', 0)
        
        # Calculate coverage for each group
        row_data = {
            'pop_label': pop_bucket,
            'total_pumas': total_pumas,
            'rural_pumas': rural_pumas,
            'urban_pumas': urban_pumas
        }
        
        # Total coverage (all areas)
        for level in naics_levels:
            total_employment = bucket_data['PERWT'].sum()
            classifiable_employment = bucket_data[bucket_data[level] != 'Unclassified']['PERWT'].sum()
            coverage_pct = (classifiable_employment / total_employment * 100) if total_employment > 0 else 0
            row_data[f'total_{level}'] = round(coverage_pct, 1)
        
        # Rural coverage
        rural_data = bucket_data[bucket_data['metro_status'] == 'Non-metro/Rural']
        for level in naics_levels:
            if len(rural_data) > 0:
                rural_employment = rural_data['PERWT'].sum()
                rural_classifiable = rural_data[rural_data[level] != 'Unclassified']['PERWT'].sum()
                rural_coverage = (rural_classifiable / rural_employment * 100) if rural_employment > 0 else 0
                row_data[f'rural_{level}'] = round(rural_coverage, 1)
            else:
                row_data[f'rural_{level}'] = 0.0
        
        # Urban coverage
        urban_data = bucket_data[bucket_data['metro_status'] == 'Metropolitan']
        for level in naics_levels:
            if len(urban_data) > 0:
                urban_employment = urban_data['PERWT'].sum()
                urban_classifiable = urban_data[urban_data[level] != 'Unclassified']['PERWT'].sum()
                urban_coverage = (urban_classifiable / urban_employment * 100) if urban_employment > 0 else 0
                row_data[f'urban_{level}'] = round(urban_coverage, 1)
            else:
                row_data[f'urban_{level}'] = 0.0
        
        coverage_breakdown.append(row_data)
    
    # Create DataFrame
    combined_detailed = pd.DataFrame(coverage_breakdown)
    
    # Sort by population bucket order
    bucket_order = ['0-75K', '75-100K', '100-125K', '125-150K', '150-175K', 
                   '175-200K', '200-250K', '250-300K', '300-400K', '400-500K', '500K+']
    combined_detailed['pop_label'] = pd.Categorical(combined_detailed['pop_label'], categories=bucket_order, ordered=True)
    combined_detailed = combined_detailed.sort_values('pop_label').reset_index(drop=True)
    
    return combined_detailed

combined_detailed_coverage = create_combined_coverage_with_split()

# STEP 4: Display Results
print("\n" + "="*80)
print("RURAL COVERAGE BY NAICS DETAIL LEVEL")
print("="*80)
if len(rural_coverage) > 0:
    print(rural_coverage.to_string(index=False))
else:
    print("No rural data available")

print("\n" + "="*80)
print("URBAN COVERAGE BY NAICS DETAIL LEVEL") 
print("="*80)
if len(urban_coverage) > 0:
    print(urban_coverage.to_string(index=False))
else:
    print("No urban data available")

print("\n" + "="*80)
print("COMBINED DETAILED COVERAGE (WITH RURAL/URBAN BREAKDOWN)")
print("="*80)
print("Shows total, rural, and urban coverage within each population bucket")

# Display key columns for readability
display_cols = ['pop_label', 'total_pumas', 'rural_pumas', 'urban_pumas', 
               'total_naics_2digit', 'rural_naics_2digit', 'urban_naics_2digit',
               'total_naics_5digit', 'rural_naics_5digit', 'urban_naics_5digit']

if all(col in combined_detailed_coverage.columns for col in display_cols):
    print("Coverage Summary (2-digit and 5-digit NAICS):")
    print(combined_detailed_coverage[display_cols].to_string(index=False))
    
    print(f"\nFull detailed breakdown with all NAICS levels available in DataFrame")
else:
    # Fallback to showing available columns
    print("Available columns:", combined_detailed_coverage.columns.tolist())
    print(combined_detailed_coverage.head())

# STEP 5: Summary comparison
print("\n" + "="*80)
print("RURAL vs URBAN COMPARISON")
print("="*80)

if len(rural_coverage) > 0 and len(urban_coverage) > 0:
    # Calculate averages
    rural_avg = rural_coverage[['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']].mean()
    urban_avg = urban_coverage[['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']].mean()
    
    comparison_df = pd.DataFrame({
        'NAICS_Level': ['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit'],
        'Rural_Avg': rural_avg.round(1).values,
        'Urban_Avg': urban_avg.round(1).values
    })
    comparison_df['Difference'] = (comparison_df['Urban_Avg'] - comparison_df['Rural_Avg']).round(1)
    
    print("Average coverage percentages:")
    print(comparison_df.to_string(index=False))
    
    # Summary stats
    rural_pumas = rural_coverage['n_pumas'].sum()
    urban_pumas = urban_coverage['n_pumas'].sum()
    total_pumas = combined_coverage['n_pumas'].sum()
    
    print(f"\nPUMA Distribution:")
    print(f"  Rural PUMAs: {rural_pumas}")
    print(f"  Urban PUMAs: {urban_pumas}")
    print(f"  Total PUMAs: {total_pumas}")

# Save results
rural_coverage.to_csv("rural_naics_coverage.csv", index=False)
urban_coverage.to_csv("urban_naics_coverage.csv", index=False)
combined_detailed_coverage.to_csv("detailed_rural_urban_coverage.csv", index=False)

print(f"\nResults saved:")
print(f"  - rural_naics_coverage.csv (rural only)")
print(f"  - urban_naics_coverage.csv (urban only)") 
print(f"  - detailed_rural_urban_coverage.csv (with rural/urban breakdown)")

print(f"\nFinal DataFrames:")
print(f"  - rural_coverage: Rural areas only")
print(f"  - urban_coverage: Urban areas only")
print(f"  - combined_detailed_coverage: Shows total + rural/urban splits")

In [63]:


# Create separate coverage analysis by metro status
def create_coverage_by_metro_status(employed_data, puma_data, metro_type):
    """Create coverage analysis for specific metro status"""
    
    # Filter data
    metro_employed = employed_data[employed_data['metro_status'] == metro_type]
    metro_pumas = puma_data[puma_data['metro_status'] == metro_type]
    
    if len(metro_employed) == 0:
        print(f"No data for {metro_type}")
        return pd.DataFrame(), pd.DataFrame()
    
    print(f"Analyzing {metro_type}: {len(metro_employed):,} employment records")
    
    # Apply detailed NAICS mapping if not already done
    if 'naics_2digit' not in metro_employed.columns:
        def create_naics_detail_levels(ind_code):
            if pd.isna(ind_code) or ind_code == 0:
                return {
                    'naics_2digit': 'Unclassified',
                    'naics_3digit': 'Unclassified',
                    'naics_4digit': 'Unclassified', 
                    'naics_5digit': 'Unclassified',
                    'naics_6digit': 'Unclassified'
                }
            
            ind_str = str(int(ind_code)).zfill(4)
            return {
                'naics_2digit': ind_str[0] + '0',
                'naics_3digit': ind_str[:2] + '0',
                'naics_4digit': ind_str[:3] + '0',
                'naics_5digit': ind_str,
                'naics_6digit': ind_str + str(ind_code % 10)
            }
  
        naics_detail_mapping = metro_employed['IND'].apply(create_naics_detail_levels)
        naics_detail_df = pd.DataFrame(naics_detail_mapping.tolist())
        
        for col in naics_detail_df.columns:
            metro_employed[col] = naics_detail_df[col]
    
    # Calculate coverage for each NAICS level
    naics_levels = ['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']
    coverage_results = {}
    
    for level in naics_levels:
        coverage_by_pop = []
        
        # Only analyze population buckets that actually exist
        existing_pop_buckets = metro_employed['pop_label'].dropna().unique()
        
        for pop_bucket in existing_pop_buckets:
            bucket_data = metro_employed[metro_employed['pop_label'] == pop_bucket]
            
            total_employment = bucket_data['PERWT'].sum()
            classifiable_employment = bucket_data[bucket_data[level] != 'Unclassified']['PERWT'].sum()
            coverage_pct = (classifiable_employment / total_employment * 100) if total_employment > 0 else 0
            
            coverage_by_pop.append({
                'pop_label': pop_bucket,
                'level': level,
                'coverage_percent': coverage_pct,
                'metro_status': metro_type
            })
        
        coverage_results[level] = pd.DataFrame(coverage_by_pop)
    
    # Combine coverage results
    all_coverage = pd.concat(coverage_results.values(), ignore_index=True)
    
    # Pivot to get coverage by level and population bucket
    coverage_pivot = all_coverage.pivot(index='pop_label', columns='level', values='coverage_percent').round(1)
    
    # Add PUMA counts (keep all population buckets, show 0 for empty ones)
    puma_counts = metro_pumas['pop_label'].value_counts().sort_index()
    
    # Create complete population bucket list (including empty ones)
    all_possible_buckets = pd.Index(['0-75K', '75-100K', '100-125K', '125-150K', '150-175K', 
                                   '175-200K', '200-250K', '250-300K', '300-400K', '400-500K', '500K+'])
    puma_counts = puma_counts.reindex(all_possible_buckets, fill_value=0)
    
    puma_counts_df = puma_counts.reset_index()
    puma_counts_df.columns = ['pop_label', 'n_pumas']
    
    # Merge and fill coverage values
    coverage_pivot = coverage_pivot.reset_index().merge(puma_counts_df, on='pop_label', how='right')
    
    # Fill NaN coverage values with 0 for population buckets with no PUMAs
    naics_levels = ['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']
    for level in naics_levels:
        coverage_pivot[level] = coverage_pivot[level].fillna(0)
    
    # Reorder columns and sort by population bucket
    cols = ['pop_label', 'n_pumas'] + naics_levels
    coverage_pivot = coverage_pivot[cols]
    
    # Sort by population bucket order
    bucket_order = ['0-75K', '75-100K', '100-125K', '125-150K', '150-175K', 
                   '175-200K', '200-250K', '250-300K', '300-400K', '400-500K', '500K+']
    coverage_pivot['pop_label'] = pd.Categorical(coverage_pivot['pop_label'], categories=bucket_order, ordered=True)
    coverage_pivot = coverage_pivot.sort_values('pop_label').reset_index(drop=True)
    
    return coverage_pivot, puma_counts

In [64]:
# Create coverage for Rural, Urban, and Combined
print("Creating coverage analysis for rural areas...")
rural_coverage, rural_puma_counts = create_coverage_by_metro_status(
    all_employed_with_pop, all_puma_pop, 'Non-metro/Rural'
)

print("Creating coverage analysis for urban areas...")
urban_coverage, urban_puma_counts = create_coverage_by_metro_status(
    all_employed_with_pop, all_puma_pop, 'Metropolitan'
)

Creating coverage analysis for rural areas...
Analyzing Non-metro/Rural: 1,766,302 employment records
Creating coverage analysis for urban areas...
Analyzing Metropolitan: 5,615,175 employment records


In [65]:
# Create combined total
print("Creating combined totals...")

# Combine PUMA counts
all_puma_counts = all_puma_pop['pop_label'].value_counts().sort_index()

# Create combined coverage analysis
def create_combined_coverage():
    """Create coverage analysis for all areas combined"""
    
    naics_levels = ['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']
    coverage_results = {}
    
    for level in naics_levels:
        coverage_by_pop = []
        
        existing_pop_buckets = all_employed_with_pop['pop_label'].dropna().unique()
        
        for pop_bucket in existing_pop_buckets:
            bucket_data = all_employed_with_pop[all_employed_with_pop['pop_label'] == pop_bucket]
            
            total_employment = bucket_data['PERWT'].sum()
            classifiable_employment = bucket_data[bucket_data[level] != 'Unclassified']['PERWT'].sum()
            coverage_pct = (classifiable_employment / total_employment * 100) if total_employment > 0 else 0
            
            coverage_by_pop.append({
                'pop_label': pop_bucket,
                'level': level,
                'coverage_percent': coverage_pct
            })
        
        coverage_results[level] = pd.DataFrame(coverage_by_pop)
    
    # Combine and pivot
    all_coverage = pd.concat(coverage_results.values(), ignore_index=True)
    coverage_pivot = all_coverage.pivot(index='pop_label', columns='level', values='coverage_percent').round(1)
    
    # Add PUMA counts (keep all population buckets, show 0 for empty ones)
    all_possible_buckets = pd.Index(['0-75K', '75-100K', '100-125K', '125-150K', '150-175K', 
                                   '175-200K', '200-250K', '250-300K', '300-400K', '400-500K', '500K+'])
    puma_counts_df = all_puma_counts.reindex(all_possible_buckets, fill_value=0).reset_index()
    puma_counts_df.columns = ['pop_label', 'n_pumas'] 
    
    coverage_pivot = coverage_pivot.reset_index().merge(puma_counts_df, on='pop_label', how='right')
    
    # Fill NaN coverage values with 0 for population buckets with no PUMAs
    naics_levels = ['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']
    for level in naics_levels:
        coverage_pivot[level] = coverage_pivot[level].fillna(0)
    
    # Reorder and sort
    cols = ['pop_label', 'n_pumas'] + naics_levels
    coverage_pivot = coverage_pivot[cols]
    
    bucket_order = ['0-75K', '75-100K', '100-125K', '125-150K', '150-175K', 
                   '175-200K', '200-250K', '250-300K', '300-400K', '400-500K', '500K+']
    coverage_pivot['pop_label'] = pd.Categorical(coverage_pivot['pop_label'], categories=bucket_order, ordered=True)
    coverage_pivot = coverage_pivot.sort_values('pop_label').reset_index(drop=True)
    
    return coverage_pivot

combined_coverage = create_combined_coverage()

Creating combined totals...


In [ ]:
# Display Results
print("\n" + "="*80)
print("RURAL COVERAGE BY NAICS DETAIL LEVEL")
print("="*80)
if len(rural_coverage) > 0:
    print(rural_coverage.to_string(index=False))
else:
    print("No rural data available")

print("\n" + "="*80)
print("URBAN COVERAGE BY NAICS DETAIL LEVEL") 
print("="*80)
if len(urban_coverage) > 0:
    print(urban_coverage.to_string(index=False))
else:
    print("No urban data available")

print("\n" + "="*80)
print("COMBINED TOTAL COVERAGE BY NAICS DETAIL LEVEL")
print("="*80)
print(combined_coverage.to_string(index=False))


In [ ]:
# Summary comparison
print("\n" + "="*80)
print("RURAL vs URBAN COMPARISON")
print("="*80)

if len(rural_coverage) > 0 and len(urban_coverage) > 0:
    # Calculate averages
    rural_avg = rural_coverage[['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']].mean()
    urban_avg = urban_coverage[['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']].mean()
    
    comparison_df = pd.DataFrame({
        'NAICS_Level': ['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit'],
        'Rural_Avg': rural_avg.round(1).values,
        'Urban_Avg': urban_avg.round(1).values
    })
    comparison_df['Difference'] = (comparison_df['Urban_Avg'] - comparison_df['Rural_Avg']).round(1)
    
    print("Average coverage percentages:")
    print(comparison_df.to_string(index=False))
    
    # Summary stats
    rural_pumas = rural_coverage['n_pumas'].sum()
    urban_pumas = urban_coverage['n_pumas'].sum()
    total_pumas = combined_coverage['n_pumas'].sum()
    
    print(f"\nPUMA Distribution:")
    print(f"  Rural PUMAs: {rural_pumas}")
    print(f"  Urban PUMAs: {urban_pumas}")
    print(f"  Total PUMAs: {total_pumas}")

# # Save results
# rural_coverage.to_csv("rural_naics_coverage.csv", index=False)
# urban_coverage.to_csv("urban_naics_coverage.csv", index=False)
# combined_coverage.to_csv("combined_naics_coverage.csv", index=False)

print(f"\nResults saved:")
print(f"  - rural_naics_coverage.csv")
print(f"  - urban_naics_coverage.csv") 
print(f"  - combined_naics_coverage.csv")

print(f"\nFinal DataFrames:")
print(f"  - rural_coverage: Rural areas only")
print(f"  - urban_coverage: Urban areas only")
print(f"  - combined_coverage: All areas together (no NaN rows)")

In [66]:
combined_coverage

,pop_label,n_pumas,naics_2digit,naics_3digit,naics_4digit,naics_5digit,naics_6digit
0,0-75K,0,0.0,0.0,0.0,0.0,0.0
1,75-100K,34,100.0,100.0,100.0,100.0,100.0
2,100-125K,1058,100.0,100.0,100.0,100.0,100.0
3,125-150K,728,100.0,100.0,100.0,100.0,100.0
4,150-175K,408,100.0,100.0,100.0,100.0,100.0
5,175-200K,199,100.0,100.0,100.0,100.0,100.0
6,200-250K,34,100.0,100.0,100.0,100.0,100.0
7,250-300K,1,100.0,100.0,100.0,100.0,100.0
8,300-400K,0,0.0,0.0,0.0,0.0,0.0
9,400-500K,0,0.0,0.0,0.0,0.0,0.0


In [56]:
# Calculate employment coverage at each detail level
print("Calculating employment coverage at each NAICS detail level...")

naics_levels = ['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']
coverage_results = {}

for level in naics_levels:
    print(f"Analyzing {level}...")
    
    # Count unique categories at this detail level
    unique_categories = all_employed_with_pop[level].nunique()
    
    # Employment by category at this level
    employment_by_category = all_employed_with_pop.groupby(['pop_label', level], observed=True)['PERWT'].sum().reset_index()
    
    # Coverage by population bucket
    coverage_by_pop = []
    
    for pop_bucket in all_employed_with_pop['pop_label'].dropna().unique():
        bucket_data = all_employed_with_pop[all_employed_with_pop['pop_label'] == pop_bucket]
        
        total_employment = bucket_data['PERWT'].sum()
        
        # Count categories with meaningful employment (>1% of bucket total)
        bucket_categories = bucket_data.groupby(level)['PERWT'].sum()
        meaningful_categories = (bucket_categories > total_employment * 0.01).sum()
        
        # Employment in classifiable categories (not "Unclassified") 
        classifiable_employment = bucket_data[bucket_data[level] != 'Unclassified']['PERWT'].sum()
        coverage_pct = (classifiable_employment / total_employment * 100) if total_employment > 0 else 0
        
        coverage_by_pop.append({
            'pop_label': pop_bucket,
            'level': level,
            'total_employment': total_employment,
            'classifiable_employment': classifiable_employment,
            'coverage_percent': coverage_pct,
            'total_categories': len(bucket_categories),
            'meaningful_categories': meaningful_categories,
            'unique_categories_overall': unique_categories
        })
    
    coverage_results[level] = pd.DataFrame(coverage_by_pop)

Calculating employment coverage at each NAICS detail level...
Analyzing naics_2digit...
Analyzing naics_3digit...
Analyzing naics_4digit...
Analyzing naics_5digit...
Analyzing naics_6digit...


In [62]:
# Create summary table of coverage by detail level
print("Creating coverage summary...")

# Combine all coverage results
all_coverage = pd.concat(coverage_results.values(), ignore_index=True)

# Pivot to get coverage by level and population bucket
coverage_pivot = all_coverage.pivot(index='pop_label', columns='level', values='coverage_percent').round(1)

# Add PUMA counts
if 'puma_counts' in locals():
    puma_counts_df = puma_counts.reset_index()
    puma_counts_df.columns = ['pop_label', 'n_pumas']
    coverage_pivot = coverage_pivot.reset_index().merge(puma_counts_df, on='pop_label', how='left')
    
    # Reorder columns
    cols = ['pop_label', 'n_pumas'] + naics_levels
    coverage_pivot = coverage_pivot[cols]
else:
    coverage_pivot = coverage_pivot.reset_index()

coverage_pivot

Creating coverage summary...


,pop_label,n_pumas,naics_2digit,naics_3digit,naics_4digit,naics_5digit,naics_6digit
0,100-125K,212.0,100.0,100.0,100.0,100.0,100.0
1,125-150K,163.0,100.0,100.0,100.0,100.0,100.0
2,150-175K,80.0,100.0,100.0,100.0,100.0,100.0
3,175-200K,45.0,100.0,100.0,100.0,100.0,100.0
4,200-250K,6.0,100.0,100.0,100.0,100.0,100.0
5,250-300K,NaN,100.0,100.0,100.0,100.0,100.0
6,75-100K,3.0,100.0,100.0,100.0,100.0,100.0


In [58]:
# Category diversity analysis
print("Category diversity at each detail level...")

diversity_summary = []
for level in naics_levels:
    level_data = coverage_results[level]
    
    avg_coverage = level_data['coverage_percent'].mean()
    avg_total_categories = level_data['total_categories'].mean()
    avg_meaningful_categories = level_data['meaningful_categories'].mean()
    overall_unique = level_data['unique_categories_overall'].iloc[0]
    
    diversity_summary.append({
        'detail_level': level,
        'avg_coverage_percent': round(avg_coverage, 1),
        'avg_categories_per_area': round(avg_total_categories, 1),
        'avg_meaningful_categories': round(avg_meaningful_categories, 1),
        'total_unique_categories': overall_unique
    })

diversity_df = pd.DataFrame(diversity_summary)

print("NAICS DETAIL LEVEL DIVERSITY ANALYSIS")
print("="*60)
print(diversity_df.to_string(index=False))
print()

print("Column definitions:")
print("- avg_coverage_percent: Average % of employment classifiable at this level")
print("- avg_categories_per_area: Average number of industry categories per area")
print("- avg_meaningful_categories: Average categories with >1% employment")
print("- total_unique_categories: Total unique industry categories found")
print()

Category diversity at each detail level...
NAICS DETAIL LEVEL DIVERSITY ANALYSIS
detail_level  avg_coverage_percent  avg_categories_per_area  avg_meaningful_categories  total_unique_categories
naics_2digit                 100.0                     10.0                        9.7                       10
naics_3digit                 100.0                     93.1                       31.3                       94
naics_4digit                 100.0                    246.7                       21.4                      255
naics_5digit                 100.0                    255.7                       21.1                      264
naics_6digit                 100.0                    255.7                       21.1                      264

Column definitions:
- avg_coverage_percent: Average % of employment classifiable at this level
- avg_categories_per_area: Average number of industry categories per area
- avg_meaningful_categories: Average categories with >1% employment
- total_u

In [59]:
# Detail level comparison by metro status
print("Coverage by metro status and detail level...")

if 'metro_status' in all_employed_with_pop.columns:
    metro_coverage = []
    
    for metro_type in all_employed_with_pop['metro_status'].unique():
        metro_data = all_employed_with_pop[all_employed_with_pop['metro_status'] == metro_type]
        
        for level in naics_levels:
            total_emp = metro_data['PERWT'].sum()
            classifiable_emp = metro_data[metro_data[level] != 'Unclassified']['PERWT'].sum()
            coverage = (classifiable_emp / total_emp * 100) if total_emp > 0 else 0
            
            metro_coverage.append({
                'metro_status': metro_type,
                'naics_level': level,
                'coverage_percent': round(coverage, 1),
                'unique_categories': metro_data[level].nunique()
            })
    
    metro_coverage_df = pd.DataFrame(metro_coverage)
    metro_pivot = metro_coverage_df.pivot(index='metro_status', columns='naics_level', values='coverage_percent')
    
    print("Coverage by Metro Status and NAICS Detail Level:")
    print(metro_pivot.to_string())
    print()

Coverage by metro status and detail level...
Coverage by Metro Status and NAICS Detail Level:
naics_level      naics_2digit  naics_3digit  naics_4digit  naics_5digit  naics_6digit
metro_status                                                                         
Metropolitan            100.0         100.0         100.0         100.0         100.0
Non-metro/Rural         100.0         100.0         100.0         100.0         100.0



In [60]:
# Expected pattern analysis (simplified)
print("Checking if coverage decreases as detail increases...")

expected_pattern = coverage_pivot[naics_levels].mean()
print("Average coverage by detail level:")
for level, avg_cov in expected_pattern.items():
    print(f"  {level}: {avg_cov:.1f}%")

# Simple pattern check
decreasing_pattern = True
for i in range(len(naics_levels)-1):
    if expected_pattern[naics_levels[i]] <= expected_pattern[naics_levels[i+1]]:
        decreasing_pattern = False
        break

print(f"\nPattern check: {decreasing_pattern}")
if decreasing_pattern:
    print("Good: Coverage decreases as detail increases (as expected)")
else:
    print("Unexpected: Some detailed levels have higher coverage")

Checking if coverage decreases as detail increases...
Average coverage by detail level:
  naics_2digit: 100.0%
  naics_3digit: 100.0%
  naics_4digit: 100.0%
  naics_5digit: 100.0%
  naics_6digit: 100.0%

Pattern check: False
Unexpected: Some detailed levels have higher coverage


In [48]:
# Create All PUMAs (Rural + Non-Rural)
print("Creating ALL PUMAs analysis (rural + non-rural)...")

# All PUMAs with PUMA identification
all_puma_df = df[df['PUMA'] != 0].copy()

# Add metro status classification
all_puma_df['metro_status'] = all_puma_df['MET2013'].apply(
    lambda x: 'Non-metro/Rural' if x == 0 else 'Metropolitan'
)

print(f"Total PUMAs data: {len(all_puma_df):,} records")
print(f"Total population: {all_puma_df['PERWT'].sum():,.0f}")

# Check metro status distribution
metro_distribution = all_puma_df.groupby('metro_status')['PERWT'].sum()
for status, pop in metro_distribution.items():
    pct = pop / metro_distribution.sum() * 100
    print(f"  {status}: {pop:,.0f} ({pct:.1f}%)")
print()

Creating ALL PUMAs analysis (rural + non-rural)...
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Total PUMAs data: 15,912,393 records
Total population: 332,387,543
  Metropolitan: 263,263,875 (79.2%)
  Non-metro/Rural: 69,123,668 (20.8%)



In [49]:
#PUMA Population Analysis for ALL PUMAs
print("PUMA population analysis for ALL areas...")

# Create unique PUMA identifier
all_puma_df['unique_puma'] = all_puma_df['STATEFIP'].astype(str) + '_' + all_puma_df['PUMA'].astype(str)

# Calculate PUMA populations by metro status
all_puma_pop = all_puma_df.groupby('unique_puma').agg({
    'PERWT': 'sum',
    'STATEFIP': 'first',
    'PUMA': 'first',
    'MET2013': 'first'
}).reset_index()
all_puma_pop.columns = ['unique_puma', 'population', 'STATEFIP', 'PUMA', 'MET2013']

# Add metro status
all_puma_pop['metro_status'] = all_puma_pop['MET2013'].apply(
    lambda x: 'Non-metro/Rural' if x == 0 else 'Metropolitan'
)

# Create population bins
pop_bins = [0, 75000, 100000, 125000, 150000, 175000, 200000, 250000, 300000, 400000, 500000, np.inf]
pop_labels = ['0-75K', '75-100K', '100-125K', '125-150K', '150-175K', '175-200K', 
              '200-250K', '250-300K', '300-400K', '400-500K', '500K+']

all_puma_pop['pop_label'] = pd.cut(all_puma_pop['population'], bins=pop_bins, labels=pop_labels, right=False)

print(f"Total PUMAs: {len(all_puma_pop)}")
print(f"Rural PUMAs: {(all_puma_pop['metro_status'] == 'Non-metro/Rural').sum()}")
print(f"Metro PUMAs: {(all_puma_pop['metro_status'] == 'Metropolitan').sum()}")
print()

# PUMAs by population size and metro status
puma_counts_by_metro = all_puma_pop.groupby(['pop_label', 'metro_status']).size().unstack(fill_value=0)
print("PUMAs by population size and metro status:")
print(puma_counts_by_metro)
print()

PUMA population analysis for ALL areas...
Total PUMAs: 2462
Rural PUMAs: 509
Metro PUMAs: 1953

PUMAs by population size and metro status:
metro_status  Metropolitan  Non-metro/Rural
pop_label                                  
0-75K                    0                0
75-100K                 31                3
100-125K               846              212
125-150K               565              163
150-175K               328               80
175-200K               154               45
200-250K                28                6
250-300K                 1                0
300-400K                 0                0
400-500K                 0                0
500K+                    0                0



/tmp/ipykernel_1389/2499526816.py:34: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  puma_counts_by_metro = all_puma_pop.groupby(['pop_label', 'metro_status']).size().unstack(fill_value=0)


In [52]:
# Employment Analysis for ALL PUMAs
print("Employment analysis for ALL PUMAs...")

# Get employed persons from all PUMAs
all_employed = all_puma_df[all_puma_df['EMPSTAT'] == 1].copy()
print(f"Total employed persons: {all_employed['PERWT'].sum():,.0f}")

# Apply NAICS mapping (same function as before)
def map_to_naics_2digit(ind_code):
    if pd.isna(ind_code) or ind_code == 0:
        return 'Other'
    
    ind_str = str(int(ind_code)).zfill(4)
    
    if ind_str.startswith(('017', '018', '019', '027', '028', '029')):
        return 'NAICS_11'
    elif ind_str.startswith(('067', '068', '069', '077', '078', '079')):
        return 'NAICS_23'
    elif ind_str.startswith(('107', '108', '109', '117', '118', '119', '127', '128', '129',
                             '137', '138', '139', '147', '148', '149', '157', '158', '159',
                             '167', '168', '169', '177', '178', '179', '187', '188', '189',
                             '197', '198', '199', '207', '208', '209', '217', '218', '219',
                             '227', '228', '229', '237', '238', '239', '247', '248', '249',
                             '257', '258', '259', '267', '268', '269', '277', '278', '279',
                             '287', '288', '289', '297', '298', '299', '307', '308', '309',
                             '317', '318', '319', '327', '328', '329', '337', '338', '339',
                             '347', '348', '349', '357', '358', '359', '367', '368', '369',
                             '377', '378', '379', '387', '388', '389', '397', '398', '399')):
        return 'NAICS_31-33'
    elif ind_str.startswith(('447', '448', '449', '457', '458', '459', '467', '468', '469',
                             '477', '478', '479', '487', '488', '489', '497', '498', '499',
                             '507', '508', '509', '517', '518', '519', '527', '528', '529',
                             '537', '538', '539', '547', '548', '549', '557', '558', '559',
                             '567', '568', '569', '577', '578', '579', '587', '588', '589',
                             '597', '598', '599')):
        return 'NAICS_44-45'
    elif ind_str.startswith(('827', '828', '829', '837', '838', '839', '847', '848', '849',
                             '857', '858', '859', '867', '868', '869')):
        return 'NAICS_62'
    else:
        return 'Other'

all_employed['naics_2digit'] = all_employed['IND'].apply(map_to_naics_2digit)
all_employed['unique_puma'] = all_employed['STATEFIP'].astype(str) + '_' + all_employed['PUMA'].astype(str)

# Drop metro_status from all_employed before merging
all_employed_clean = all_employed.drop(columns=['metro_status'])

# Then merge
all_employed_with_pop = all_employed_clean.merge(
    all_puma_pop[['unique_puma', 'pop_label', 'metro_status']], 
    on='unique_puma', how='left'
)
print("Employment by metro status:")
emp_by_metro = all_employed_with_pop.groupby('metro_status')['PERWT'].sum()
for status, emp in emp_by_metro.items():
    pct = emp / emp_by_metro.sum() * 100
    print(f"  {status}: {emp:,.0f} ({pct:.1f}%)")
print()

Employment analysis for ALL PUMAs...
Total employed persons: 160,913,396
Employment by metro status:
  Metropolitan: 129,714,051 (80.6%)
  Non-metro/Rural: 31,199,345 (19.4%)



In [53]:
# Create Separate Analyses
print("Creating separate rural and metro analyses...")

# A. RURAL ONLY Analysis
rural_employed_with_pop = all_employed_with_pop[all_employed_with_pop['metro_status'] == 'Non-metro/Rural']
rural_puma_pop = all_puma_pop[all_puma_pop['metro_status'] == 'Non-metro/Rural']

# B. METRO ONLY Analysis  
metro_employed_with_pop = all_employed_with_pop[all_employed_with_pop['metro_status'] == 'Metropolitan']
metro_puma_pop = all_puma_pop[all_puma_pop['metro_status'] == 'Metropolitan']

# C. COMBINED Analysis by Metro Status
def create_employment_summary(employed_data, puma_data, analysis_name):
    """Create employment summary for a given dataset"""
    
    if len(employed_data) == 0:
        print(f"No data for {analysis_name}")
        return pd.DataFrame()
    
    # Calculate employment by population bucket and NAICS sector
    employment_by_pop_naics = employed_data.groupby(['pop_label', 'naics_2digit'], observed=True)['PERWT'].sum().unstack(fill_value=0)
    
    # Calculate employment percentages
    total_employment_by_pop = employment_by_pop_naics.sum(axis=1)
    employment_percentages = employment_by_pop_naics.div(total_employment_by_pop, axis=0) * 100
   
    # Population summary
    population_summary = puma_data.groupby('pop_label', observed=True).agg({
        'population': 'sum',
        'unique_puma': 'count'
    }).rename(columns={'unique_puma': 'n_pumas'}).reset_index()
    
    employed_summary = employed_data.groupby('pop_label', observed=True)['PERWT'].sum().reset_index()
    employed_summary.columns = ['pop_label', 'total_employed']
    
    pop_emp_summary = population_summary.merge(employed_summary, on='pop_label', how='left')
    pop_emp_summary['total_employed'] = pop_emp_summary['total_employed'].fillna(0)
    
    # Create complete summary
    all_sectors = [col for col in employment_percentages.columns if col.startswith('NAICS') or col == 'Other']
    complete_summary = employment_percentages[all_sectors].fillna(0).round(0).astype(int)
    complete_summary = complete_summary.reset_index()
    complete_summary = complete_summary.merge(pop_emp_summary, on='pop_label', how='left')
    
    # Reorder columns
    key_cols = ['pop_label', 'n_pumas', 'population', 'total_employed'] 
    sector_cols = [col for col in complete_summary.columns if col.startswith('NAICS') or col == 'Other']
    complete_summary = complete_summary[key_cols + sector_cols]
    
    return complete_summary

# Create all three analyses
print("Creating employment summaries...")
rural_summary = create_employment_summary(rural_employed_with_pop, rural_puma_pop, "Rural")
metro_summary = create_employment_summary(metro_employed_with_pop, metro_puma_pop, "Metro") 
all_puma_summary = create_employment_summary(all_employed_with_pop, all_puma_pop, "All PUMAs")

Creating separate rural and metro analyses...
Creating employment summaries...


In [54]:
rural_summary

,pop_label,n_pumas,population,total_employed,NAICS_11,NAICS_23,NAICS_31-33,NAICS_44-45,NAICS_62,Other
0,75-100K,3,299001.0,117644.0,8,9,5,12,12,54
1,100-125K,212,23947937.0,10832676.0,3,8,13,12,12,51
2,125-150K,163,22141412.0,10003378.0,3,8,13,12,12,52
3,150-175K,80,13052213.0,5901389.0,3,8,12,12,12,53
4,175-200K,45,8399756.0,3752598.0,2,8,12,12,12,55
5,200-250K,6,1283349.0,591660.0,2,8,8,12,12,57


In [ ]:


# STEP 1: Create detailed NAICS mappings from IND codes
print("\nStep 1: Creating detailed NAICS mappings...")

def create_naics_detail_levels(ind_code):
    """
    Create different levels of NAICS detail from IND codes
    This simulates increasing detail levels
    """
    if pd.isna(ind_code) or ind_code == 0:
        return {
            'naics_2digit': 'Unclassified',
            'naics_3digit': 'Unclassified',
            'naics_4digit': 'Unclassified', 
            'naics_5digit': 'Unclassified',
            'naics_6digit': 'Unclassified'
        }
    
    # Convert to string for digit extraction
    ind_str = str(int(ind_code)).zfill(4)
    
    # Create progressively more detailed NAICS codes
    # This simulates how detail increases with more digits
    return {
        'naics_2digit': ind_str[0] + '0',           # First digit + 0 (e.g., '10', '20', '30')
        'naics_3digit': ind_str[:2] + '0',          # First 2 digits + 0 (e.g., '110', '230') 
        'naics_4digit': ind_str[:3] + '0',          # First 3 digits + 0 (e.g., '1110', '2380')
        'naics_5digit': ind_str,                    # All 4 digits (e.g., '1111', '2381')
        'naics_6digit': ind_str + str(ind_code % 10) # Add variation for 6-digit
    }

# Apply to all employed data (assuming all_employed_with_pop exists)
print("Applying NAICS detail mapping to employment data...")

# Get the detailed NAICS mappings
naics_detail_mapping = all_employed_with_pop['IND'].apply(create_naics_detail_levels)
naics_detail_df = pd.DataFrame(naics_detail_mapping.tolist())

# Add detail columns to employment data
for col in naics_detail_df.columns:
    all_employed_with_pop[col] = naics_detail_df[col]

print("Sample of NAICS detail levels:")
sample_cols = ['IND', 'naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit']
print(all_employed_with_pop[sample_cols].head(10))
print()

# STEP 2: Calculate employment coverage at each detail level
print("Step 2: Calculating employment coverage at each NAICS detail level...")

naics_levels = ['naics_2digit', 'naics_3digit', 'naics_4digit', 'naics_5digit', 'naics_6digit']
coverage_results = {}

for level in naics_levels:
    print(f"Analyzing {level}...")
    
    # Count unique categories at this detail level
    unique_categories = all_employed_with_pop[level].nunique()
    
    # Employment by category at this level
    employment_by_category = all_employed_with_pop.groupby(['pop_label', level], observed=True)['PERWT'].sum().reset_index()
    
    # Coverage by population bucket
    coverage_by_pop = []
    
    for pop_bucket in all_employed_with_pop['pop_label'].dropna().unique():
        bucket_data = all_employed_with_pop[all_employed_with_pop['pop_label'] == pop_bucket]
        
        total_employment = bucket_data['PERWT'].sum()
        
        # Count categories with meaningful employment (>1% of bucket total)
        bucket_categories = bucket_data.groupby(level)['PERWT'].sum()
        meaningful_categories = (bucket_categories > total_employment * 0.01).sum()
        
        # Employment in classifiable categories (not "Unclassified") 
        classifiable_employment = bucket_data[bucket_data[level] != 'Unclassified']['PERWT'].sum()
        coverage_pct = (classifiable_employment / total_employment * 100) if total_employment > 0 else 0
        
        coverage_by_pop.append({
            'pop_label': pop_bucket,
            'level': level,
            'total_employment': total_employment,
            'classifiable_employment': classifiable_employment,
            'coverage_percent': coverage_pct,
            'total_categories': len(bucket_categories),
            'meaningful_categories': meaningful_categories,
            'unique_categories_overall': unique_categories
        })
    
    coverage_results[level] = pd.DataFrame(coverage_by_pop)

# STEP 3: Create summary table of coverage by detail level
print("\nStep 3: Creating coverage summary...")

# Combine all coverage results
all_coverage = pd.concat(coverage_results.values(), ignore_index=True)

# Pivot to get coverage by level and population bucket
coverage_pivot = all_coverage.pivot(index='pop_label', columns='level', values='coverage_percent').round(1)

# Add PUMA counts
if 'puma_counts' in locals():
    puma_counts_df = puma_counts.reset_index()
    puma_counts_df.columns = ['pop_label', 'n_pumas']
    coverage_pivot = coverage_pivot.reset_index().merge(puma_counts_df, on='pop_label', how='left')
    
    # Reorder columns
    cols = ['pop_label', 'n_pumas'] + naics_levels
    coverage_pivot = coverage_pivot[cols]
else:
    coverage_pivot = coverage_pivot.reset_index()

print("EMPLOYMENT COVERAGE BY NAICS DETAIL LEVEL")
print("="*70)
print("Percentage of employment that can be classified at each detail level")
print(coverage_pivot.to_string(index=False))
print()

# STEP 4: Category diversity analysis
print("Step 4: Category diversity at each detail level...")

diversity_summary = []
for level in naics_levels:
    level_data = coverage_results[level]
    
    avg_coverage = level_data['coverage_percent'].mean()
    avg_total_categories = level_data['total_categories'].mean()
    avg_meaningful_categories = level_data['meaningful_categories'].mean()
    overall_unique = level_data['unique_categories_overall'].iloc[0]
    
    diversity_summary.append({
        'detail_level': level,
        'avg_coverage_percent': round(avg_coverage, 1),
        'avg_categories_per_area': round(avg_total_categories, 1),
        'avg_meaningful_categories': round(avg_meaningful_categories, 1),
        'total_unique_categories': overall_unique
    })

diversity_df = pd.DataFrame(diversity_summary)

print("NAICS DETAIL LEVEL DIVERSITY ANALYSIS")
print("="*60)
print(diversity_df.to_string(index=False))
print()

print("Column definitions:")
print("- avg_coverage_percent: Average % of employment classifiable at this level")
print("- avg_categories_per_area: Average number of industry categories per area")
print("- avg_meaningful_categories: Average categories with >1% employment")
print("- total_unique_categories: Total unique industry categories found")
print()

# STEP 5: Detail level comparison by metro status
print("Step 5: Coverage by metro status and detail level...")

if 'metro_status' in all_employed_with_pop.columns:
    metro_coverage = []
    
    for metro_type in all_employed_with_pop['metro_status'].unique():
        metro_data = all_employed_with_pop[all_employed_with_pop['metro_status'] == metro_type]
        
        for level in naics_levels:
            total_emp = metro_data['PERWT'].sum()
            classifiable_emp = metro_data[metro_data[level] != 'Unclassified']['PERWT'].sum()
            coverage = (classifiable_emp / total_emp * 100) if total_emp > 0 else 0
            
            metro_coverage.append({
                'metro_status': metro_type,
                'naics_level': level,
                'coverage_percent': round(coverage, 1),
                'unique_categories': metro_data[level].nunique()
            })
    
    metro_coverage_df = pd.DataFrame(metro_coverage)
    metro_pivot = metro_coverage_df.pivot(index='metro_status', columns='naics_level', values='coverage_percent')
    
    print("Coverage by Metro Status and NAICS Detail Level:")
    print(metro_pivot.to_string())
    print()

# STEP 6: Expected pattern analysis
print("Step 6: Analysis of expected patterns...")

expected_pattern = coverage_pivot[naics_levels].mean()
print("Expected pattern: Coverage should decrease as detail increases")
print("Average coverage across all areas:")
for level, avg_cov in expected_pattern.items():
    print(f"  {level}: {avg_cov:.1f}%")

print()

# Check if pattern holds
pattern_check = []
for i in range(len(naics_levels)-1):
    current_level = naics_levels[i]
    next_level = naics_levels[i+1]
    
    current_avg = expected_pattern[current_level]
    next_avg = expected_pattern[next_level]
    
    decreases = current_avg > next_avg
    pattern_check.append(decreases)
    
    print(f"{current_level} > {next_level}: {decreases} ({current_avg:.1f}% vs {next_avg:.1f}%)")

all_decreasing = all(pattern_check)
print(f"\nPattern holds (each level > next level): {all_decreasing}")

if all_decreasing:
    print("✓ Expected pattern confirmed: More detailed levels have lower coverage")
else:
    print("⚠ Unexpected pattern: Some detailed levels have higher coverage than broader levels")

# STEP 7: Save results
print("\nStep 7: Saving results...")

coverage_pivot.to_csv("naics_detail_coverage_by_population.csv", index=False)
diversity_df.to_csv("naics_detail_diversity_analysis.csv", index=False)

if 'metro_coverage_df' in locals():
    metro_coverage_df.to_csv("naics_detail_coverage_by_metro_status.csv", index=False)

print("Results saved:")
print("  - naics_detail_coverage_by_population.csv (main coverage table)")
print("  - naics_detail_diversity_analysis.csv (diversity analysis)")
if 'metro_coverage_df' in locals():
    print("  - naics_detail_coverage_by_metro_status.csv (rural vs urban coverage)")

print(f"\n" + "="*80)
print("NAICS DETAIL LEVEL ANALYSIS COMPLETE")
print("="*80)
print("Key Findings:")
print("1. Coverage percentages show what % of employment can be classified at each detail level")
print("2. Diversity shows how many industry categories exist at each level")
print("3. Pattern analysis confirms if finer detail levels have expected lower coverage")
print("4. Rural vs urban comparison shows geographic differences in industry detail")

print(f"\nFinal DataFrames:")
print(f"- coverage_pivot: Coverage by population bucket and NAICS detail level")
print(f"- diversity_df: Summary of category diversity at each detail level")
if 'metro_coverage_df' in locals():
    print(f"- metro_coverage_df: Coverage by metro status and detail level")